<a href="https://colab.research.google.com/github/brenda00/postechaiscientist/blob/main/Fase02_Aula01_Pipelines_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Laboratório Prático: Pipeline ETL & Data Quality**


Objetivo: Construir um pipeline de dados ponta-a-ponta, transformando dados "sujos" em um dataset pronto para treinamento de Machine Learning, utilizando a Arquitetura Medalhão (Bronze, Silver, Gold).

**1: Configuração do Ambiente**

Primeiro, vamos instalar a biblioteca Faker para gerar dados sintéticos e importar o necessário.

Desafio: Você pode consumir alguma API de dado ou até mesmo utilizar um outro arquivo.

In [ ]:
# Instalação de bibliotecas necessárias
!pip install pandas faker pyarrow fastparquet

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
import os

# Configuração inicial
fake = Faker('pt_BR')
Faker.seed(42)
random.seed(42)

# Criando diretórios para organizar as camadas
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/bronze', exist_ok=True)
os.makedirs('data/silver', exist_ok=True)
os.makedirs('data/gold', exist_ok=True)

print("Ambiente configurado e pastas criadas!")

**2. Geração dos Dados (Camada RAW)**

Nesta etapa, criaremos um dataset sintético de vendas. Para tornar o cenário realista, vamos inserir propositalmente alguns erros (nulos, datas futuras, valores negativos e duplicatas).

In [ ]:
def gerar_dados_raw(num_registros=1000):
    dados = []

    for _ in range(num_registros):
        # Simulando dados com erros ocasionais
        record = {
            'id_pedido': fake.unique.random_int(min=1, max=10000),
            'data_pedido': fake.date_between(start_date='-1y', end_date='+10d'), # +10d gera erro de data futura
            'cliente_nome': fake.name() if random.random() > 0.05 else None, # 5% de nulos
            'cliente_email': fake.email() if random.random() > 0.05 else "email_invalido.com", # 5% de e-mails ruins
            'categoria': random.choice(['Eletrônicos', 'Roupas', 'Livros', 'Casa']),
            'valor_total': round(random.uniform(-50, 500), 2), # Gera valores negativos propositalmente
            'estado_entrega': fake.state_abbr()
        }
        dados.append(record)

    # Criando DataFrame
    df = pd.DataFrame(dados)

    # Adicionando duplicidade proposital (duplicando 5% dos dados)
    df_duplicado = df.sample(frac=0.05)
    df_final = pd.concat([df, df_duplicado])

    # Salvando como CSV (Simulando o arquivo que chegou do sistema)
    caminho_arquivo = 'data/raw/vendas_raw.csv'
    df_final.to_csv(caminho_arquivo, index=False)
    print(f"Arquivo RAW gerado em: {caminho_arquivo} com {len(df_final)} registros.")
    return df_final

# Executando a geração
df_raw = gerar_dados_raw()
df_raw.head()

**3. Ingestão na Camada Bronze**

Na camada Bronze, o objetivo é apenas ingerir o dado bruto, converter para um formato otimizado (Parquet) e adicionar metadados de rastreabilidade (quando o dado foi processado). Não fazemos limpeza aqui.

In [ ]:
def ingestao_bronze():
    # 1. Leitura do dado bruto (RAW)
    df = pd.read_csv('data/raw/vendas_raw.csv')

    # 2. Adição de Metadados
    df['_data_ingestao'] = datetime.now()
    df['_origem'] = 'sistema_vendas_csv'

    # 3. Salvamento em formato Colunar (Parquet)
    caminho_bronze = 'data/bronze/vendas_bronze.parquet'
    df.to_parquet(caminho_bronze, index=False)

    print(f"Ingestão Bronze concluída. Arquivo salvo em: {caminho_bronze}")
    return df

# Executando ingestão
df_bronze = ingestao_bronze()

**4. Validação de Qualidade e Camada Silver**

Aplicação das regras de qualidade.

* Dados válidos vão para a tabela Silver (limpos e deduplicados).
* Dados inválidos vão para a Quarentena (para análise posterior).

In [ ]:
def processamento_silver(df):
    print(f"Total de registros recebidos da Bronze: {len(df)}")

    # --- 1. Regras de Qualidade ---

    # Regra 1: Campos obrigatórios não podem ser nulos (Nome e ID)
    regra_nulos = df['cliente_nome'].notna() & df['id_pedido'].notna()

    # Regra 2: Valor total deve ser maior que zero
    regra_valor = df['valor_total'] > 0

    # Regra 3: Validação simples de e-mail (deve conter '@')
    regra_email = df['cliente_email'].str.contains('@', na=False)

    # Regra 4: Data do pedido não pode ser futura (maior que hoje)
    hoje = datetime.now().strftime('%Y-%m-%d')
    regra_data = df['data_pedido'] <= hoje

    # Combinando todas as regras (Aprovado se passar em TODAS)
    criterio_aprovacao = regra_nulos & regra_valor & regra_email & regra_data

    # --- 2. Separação (Split) ---

    # Dados Inválidos (Quarentena)
    df_quarentena = df[~criterio_aprovacao].copy()

    # Dados Válidos (Candidatos a Silver)
    df_clean = df[criterio_aprovacao].copy()

    # --- 3. Tratamento Final (Limpeza e Padronização) ---

    # Remover duplicidades exatas nos dados válidos
    df_clean = df_clean.drop_duplicates(subset=['id_pedido'])

    # Normalização de strings (ex: Estados para maiúsculo, remover espaços)
    df_clean['estado_entrega'] = df_clean['estado_entrega'].str.upper().str.strip()

    # --- 4. Salvamento ---
    df_clean.to_parquet('data/silver/vendas_silver.parquet', index=False)
    df_quarentena.to_csv('data/silver/vendas_quarentena.csv', index=False)

    print("-" * 30)
    print(f"Registros enviados para Quarentena (Rejeitados): {len(df_quarentena)}")
    print(f"Registros aprovados e salvos na Silver: {len(df_clean)}")

    return df_clean, df_quarentena

# Executando processamento Silver
df_silver, df_quarentena = processamento_silver(df_bronze)

In [ ]:
print("Amostra de dados na Quarentena (com erros):")
display(df_quarentena.head())

**5. Camada Gold (Agregações de Negócio)**

Agora que temos dados confiáveis na Silver, criamos a camada Gold. Esta camada contém dados prontos para Dashboards e IA. Vamos criar uma visão agregada de vendas por Estado e Categoria.

In [ ]:
def geracao_gold(df_silver):
    # Agregação 1: Receita Total e Ticket Médio por Estado
    df_gold_estado = df_silver.groupby('estado_entrega').agg(
        total_vendas=('valor_total', 'sum'),
        ticket_medio=('valor_total', 'mean'),
        qtd_pedidos=('id_pedido', 'count')
    ).reset_index().sort_values(by='total_vendas', ascending=False)

    # Agregação 2: Performance por Categoria
    df_gold_categoria = df_silver.groupby('categoria').agg(
        total_vendas=('valor_total', 'sum')
    ).reset_index()

    # Salvando tabelas Gold
    df_gold_estado.to_parquet('data/gold/kpi_vendas_estado.parquet', index=False)
    df_gold_categoria.to_parquet('data/gold/kpi_vendas_categoria.parquet', index=False)

    print("Camada Gold gerada com sucesso!")
    return df_gold_estado

# Executando Gold
df_gold = geracao_gold(df_silver)

**6. Consumo Final**

Para finalizar o laboratório, vamos visualizar o resultado final que seria entregue a um cientista de dados ou analista de BI.

In [ ]:
# Simulando um analista consumindo o dado pronto
print("=== Relatório Final de Vendas por Estado (GOLD) ===")
display(df_gold.head(10))

# Validação simples
print(f"\nVerificação: O ticket médio mais alto é de R$ {df_gold['ticket_medio'].max():.2f}")